# Prediction Analysis for WSL

Lukas Graz  
February 13, 2025

In [ ]:
source("R/data_prep.R")


Number of matches per filter criteria (not disjoint)
  Headphone  PRS_all_NA    Distance Activity_NA    Duration  HMNoise_NA 
        303         226         221         102          96          96 
JourneyTime 
         20 
Keep  1494 of 2206 observations

Imputing PRS_orig_vars

## Setup

In [ ]:
Mediator_vars_noLOC <- grep("LOC_", Mediator_vars, invert = TRUE, value = TRUE)


In [ ]:
library(mlr3verse, quietly = TRUE)

mse <- msrs(c("regr.mse"))

if (!interactive())
  lgr::get_logger("mlr3")$set_threshold("warn")

get_benchi_table <- function(tasks, nfolds = 5) {
  set.seed(123)
  learners <- lrns(c("regr.featureless", "regr.lm", "regr.xgboost", "regr.ranger"))
  learners$regr.xgboost$param_set$set_values(
    eta = 0.03, 
    nrounds = 300, 
    max_depth = 2
  )

  benchi <- xfun::cache_rds({
    benchmark(benchmark_grid(
      tasks, 
      learners, 
      rsmp("cv", folds = nfolds)
    ))
  }, 
  file = "benchmark.rds", 
  dir = "cache/",
  hash = list(tasks, nfolds)
  )
  
  res <- tidyr::pivot_wider(benchi$aggregate(mse), 
    id_cols = task_id,
    names_from = learner_id,
    values_from = regr.mse
  ) |> as.data.frame()
  
  rownames(res) <- res$task_id
  res <- res[, -1]
  colnames(res) <- gsub("regr.", "", colnames(res))
  stopifnot(any(colnames(res) == "featureless"))
  res <- 1 - res / res$featureless
  res[, -1, drop = FALSE] |> round(3)
}


Testing prediction quality of GIS_vars -\> Mediators -\> PRS_vars using

-   Linear models
-   Random forests (default parameters)
-   XGBoost (with parameter tuning)
-   FASSO (not shown since inferior)

**GIS Variables:**

In [ ]:
GIS_vars


 [1] "LCARTIF_sqrt"   "LCFOREST_sqrt"  "HETER"          "OVDIST_sqrt"   
 [5] "VIS5K_sqrt"     "RL_NDVI"        "RL_NOISE"       "DISTKM_sqrt"   
 [9] "JNYTIME_sqrt"   "STRIMP123_sqrt" "STRIMP999_sqrt"

**Mediators:**

In [ ]:
Mediator_vars


[1] "FEELNAT"  "LNOISE"   "LOC_SENS" "LOC_SOUN" "LOC_SCEN" "LOC_VISE" "LOC_VEGE"
[8] "LOC_FAUN"

**Mediators (no sensory LOC variables):**

In [ ]:
Mediator_vars_noLOC


[1] "FEELNAT" "LNOISE" 

### PRS ~ GIS

In [ ]:
tasks_GIS <- lapply(PRS_vars, \(y) 
  as_task_regr(
    subset(Dmlr, select = c(y, GIS_vars)),
    target = y,
    id = y
  ))
get_benchi_table(tasks_GIS) 


         lm xgboost ranger
MEAN -0.002  -0.003 -0.031
FA   -0.002   0.015  0.000
BA   -0.006  -0.009 -0.023
EC    0.001  -0.027 -0.036
ES    0.048   0.048  0.030

GIS variables alone show poor predictive performance.

### PRS ~ GIS + Mediators

In [ ]:
tasks_GIS_MED <- lapply(PRS_vars, \(y) 
  as_task_regr(
    subset(Dmlr, select = c(y, Mediator_vars, GIS_vars)),
    target = y,
    id = y
  ))
get_benchi_table(tasks_GIS_MED) 


        lm xgboost ranger
MEAN 0.226   0.238  0.223
FA   0.236   0.263  0.254
BA   0.122   0.133  0.133
EC   0.041   0.018  0.021
ES   0.151   0.169  0.160

### PRS ~ GIS + Mediators (no sensory LOC variables)

In [ ]:
tasks_GIS_MED <- lapply(PRS_vars, \(y) 
  as_task_regr(
    subset(Dmlr, select = c(y, Mediator_vars_noLOC, GIS_vars)),
    target = y,
    id = y
  ))
get_benchi_table(tasks_GIS_MED) 


        lm xgboost ranger
MEAN 0.131   0.137  0.139
FA   0.133   0.166  0.145
BA   0.071   0.064  0.065
EC   0.017  -0.014 -0.013
ES   0.130   0.145  0.118

### PRS ~ Mediators

In [ ]:
tasks_MED <- lapply(PRS_vars, \(y) 
  as_task_regr(
    subset(Dmlr, select = c(y, Mediator_vars)),
    target = y,
    id = y
  ))
get_benchi_table(tasks_MED) 


        lm xgboost ranger
MEAN 0.225   0.237  0.193
FA   0.227   0.258  0.233
BA   0.134   0.150  0.111
EC   0.035   0.021 -0.020
ES   0.130   0.133  0.080

### PRS ~ Mediators (no sensory LOC variables)

In [ ]:
tasks_MED <- lapply(PRS_vars, \(y) 
  as_task_regr(
    subset(Dmlr, select = c(y, Mediator_vars_noLOC)),
    target = y,
    id = y
  ))
get_benchi_table(tasks_MED) 


        lm xgboost ranger
MEAN 0.134   0.148  0.137
FA   0.130   0.173  0.154
BA   0.084   0.083  0.077
EC   0.007  -0.002 -0.004
ES   0.114   0.108  0.097

### Mediators ~ GIS

In [ ]:
tasks_MED_by_GIS <- lapply(Mediator_vars, \(y) 
  as_task_regr(
    subset(Dmlr, select = c(y, GIS_vars)),
    target = y,
    id = y
  ))
get_benchi_table(tasks_MED_by_GIS)


            lm xgboost ranger
FEELNAT  0.126   0.138  0.113
LNOISE   0.097   0.072  0.084
LOC_SENS 0.020  -0.005 -0.023
LOC_SOUN 0.041   0.008 -0.001
LOC_SCEN 0.039   0.050  0.025
LOC_VISE 0.008  -0.026 -0.046
LOC_VEGE 0.052   0.029  0.035
LOC_FAUN 0.058   0.063  0.046

### Legacy Code

In [ ]:

# Get parameter estimates for XGBoost
t <- as_task_regr(
  subset(Dmlr, select = c("FEELNAT", GIS_vars)),
  target = "FEELNAT"
)

l <- lrn("regr.xgboost",
  nrounds = 500  # More iterations due to lower learning rate
)

# Create search space
ps <- ps(
  max_depth = p_int(2, 3),
  eta = p_dbl(0.001, 0.3, tags = "logscale")
)

# Setup tuning
instance <- ti(
  task = t,
  learner = l,
  resampling = rsmp("cv", folds = 3),
  measure = msr("regr.mse"),
  terminator = trm("none"),
  search_space = ps
)

# Grid search
tuner <- mlr3tuning::tnr("grid_search")
tuner$optimize(instance)
instance$result


In [ ]:

library(randomForest)

fit <- lm(as.formula(paste0(
    "cbind(", paste(PRS_vars, collapse = ", "), ")",
    " ~ ",
    paste(Mediator_vars, collapse = " + ")
  )), 
  D)
coef(fit) |> round(2)

rsq.lm <- sapply(summary(fit), \(x) x$r.sq)
rsq.rf <- sapply(PRS_vars, \(x) {
  rf <- randomForest(as.formula(paste0(
    x, " ~ ", paste(Mediator_vars, collapse = " + ")
  )), 
  D, na.action = na.omit
  ) 
  rf$rsq[500]
})

cbind(lm = rsq.lm, rf = rsq.rf) |> round(2)


In [ ]:

autoplot(mytsk1, type = "pairs")
mytsk1 <- as_task_regr(
  subset(Dmlr, select = c("FA", Mediator_vars, GIS_vars)),
  feature = c(Mediator_vars, GIS_vars),
  target = "FA",
  id = "bla"
)

lrn_xgb <- lrn("regr.xgboost")
lrn_avg <- lrn("regr.featureless")
splits <- partition(mytsk1)
lrn_xgb$train(mytsk1, splits$train)$predict(mytsk1, splits$test)$score(mse)
lrn_avg$train(mytsk1, splits$train)$predict(mytsk1, splits$test)$score(mse)
rr <- resample(mytsk1, lrn_xgb, cv3)
rr$aggregate(mse)

learners <- lrns(c("regr.featureless", "regr.lm", "regr.xgboost", "regr.ranger"))
learners$regr.xgboost$param_set$set_values(eta = 0.03, nrounds = 300, max_depth = 2)
learners <- c("regr.featureless", "regr.lm")
